# Création fichier csv clean

Ce notebook est consacré à la création du fichier qui sera utilisé comme base de données. Pour cela, nou savons pris le format CSV, avec comme séparateur la barre verticale "|" 
Il est séparé en plusieurs étapes : 

    1) Installations et importations
    2) Fonctions utilitaires
    3) Script permettant l'écriture du fichier
    

### 1) Installations et importations {#petit1}

Installations pour la récupération des données


In [ ]:
#Installations pour la récupération des données

%pip install webdriver-manager
%pip install -q lxml
%pip install bs4

Installations de spacy et du package des mots français inutiles pour le cleaning des questions

In [ ]:
#Installation pour le cleaning

%pip install spacy 
%python -m spacy download fr_core_news_sm

In [11]:
#imports

import pandas as pd
import bs4
import lxml
import pandas
import urllib
import re

from urllib import request

import spacy

### 2) Fonctions utilitaires

In [28]:
#Fonctions utilitaires

def pageurl(x) : 
    return "https://questions.assemblee-nationale.fr/q16/16-"+str(x)+"QE.htm"


nlp = spacy.load('fr_core_news_sm')

def clean_docs(texts, ministere, remove_stopwords=True, n_process = 4):
    
    docs = nlp.pipe(texts, 
                    n_process=n_process,
                    disable=['parser', 'ner', 'textcat'])
    stopwords = nlp.Defaults.stop_words.union(set(['m.','mme','ministre', ministere.lower()]))

    docs_cleaned = []
    for doc in docs:
        tokens = [tok.text.lower().strip() for tok in doc if not tok.is_punct]
        if remove_stopwords:
            tokens = [tok for tok in tokens if tok not in stopwords]
        doc_clean = ' '.join(tokens)
        docs_cleaned.append(doc_clean)
        
    return docs_cleaned

# pas sur de l'utilité : mots qui peuvent etre quand meme utile (souveraineté ?), appelle parfois le délégué des finances alors qu'interroge le ministere des comptes publics
"""def clean_question2(text, ministere, nom, remove_stopwords=True, n_process=4):
    
    text = nlp(text)

    inclus = ministere.lower().split() + nom.lower().split()
    print(inclus)
    stopwords = nlp.Defaults.stop_words 
    stopwords = stopwords.union(set(['m.','mme','ministre']+inclus))
    
    tokens = [tok.text.lower().strip() for tok in text if not tok.is_punct]
    if remove_stopwords:
        tokens = [tok for tok in tokens if tok not in stopwords]
    question_clean = ' '.join(tokens)
    return question_clean"""

def clean_question(text, nom, remove_stopwords=True, n_process=4):
    
    text = nlp(text)

    stopwords = nlp.Defaults.stop_words 
    stopwords = stopwords.union(set(nom.lower().split()+nom.split()+['m.','mme','ministre','été', 'cas',"redemande","interroge", "demande", "luire", "année", "faire", "nombre","ailleurs","dernier","concerner","situation","dispositif","aujourd'hui","part","pouvoir","permettre",'ans',"jour","appeller", "attention", "attirer", "non", "pourtant"]))
    
    tokens = [tok.lemma_ for tok in text if not tok.is_punct]
    if remove_stopwords:
        tokens = [tok for tok in tokens if tok not in stopwords]
    question_clean = ' '.join(tokens)
    return question_clean



In [29]:
#création fichier

données = open("data_cleaned_nuit.csv", "w", encoding="utf-8")

données.write("id|groupe|nom|ministère_interrogé|rubrique|titre|date|question|question_clean")

données.close()

In [38]:
données = open("data_cleaned_nuit.csv", "a", encoding="utf-8")

for k in range(13750, 14248) : 
    
    source_code = request.urlopen(pageurl(k)).read()
    page = bs4.BeautifulSoup(source_code)
    question0 = page.find("div", {"class" : "question"})
    question = question0.find("p")
    question = question.text.strip()
    
    rubrique0 = page.findAll("div", {"class" : "question_col33"})
    rubrique,titre = rubrique0[0].find("p"), rubrique0[1].find("p")
    rubrique, titre = rubrique.text.strip()[11:], titre.text.strip()[8:]
    
    ministere0 = page.find("div", {"class" : "question_col50"})
    ministere = ministere0.text.strip()[23:]

    lien0 = page.find("span", {"class" : "question_big_content"})
    lien = lien0.find("a").get("href")

    date0 = page.find("div", {"class" : "question_publish_date"})
    date = date0.find("span")
    date = date.text.strip()
    
    source_code_parlementaire = request.urlopen(lien).read()
    page_parlementaire = bs4.BeautifulSoup(source_code_parlementaire)

    groupe0 = page_parlementaire.find("a", {"class" : "h4 _colored link"})
    groupe = groupe0.text.strip()

    nom0 = page_parlementaire.find("h1", {"class" : "h1 _mt-small"})
    nom = nom0.text.strip()

    question_clean = clean_question(question, nom)
    print(k)

    données.write("\n" + str(k) + "|" + groupe + "|" + nom +  "|" + ministere + "|" + rubrique + "|" + titre + "|" + date + "|" + question + "|" + question_clean)

données.close()

13750
13751
13752
13753
13754
13755
13756
13757
13758
13759
13760
13761
13762
13763
13764
13765
13766
13767
13768
13769
13770
13771
13772
13773
13774
13775
13776
13777
13778
13779
13780
13781
13782
13783
13784
13785
13786
13787
13788
13789
13790
13791
13792
13793
13794
13795
13796
13797
13798
13799
13800
13801
13802
13803
13804
13805
13806
13807
13808
13809
13810
13811
13812
13813
13814
13815
13816
13817
13818
13819
13820
13821
13822
13823
13824
13825
13826
13827
13828
13829
13830
13831
13832
13833
13834
13835
13836
13837
13838
13839
13840
13841
13842
13843
13844
13845
13846
13847
13848
13849
13850
13851
13852
13853
13854
13855
13856
13857
13858
13859
13860
13861
13862
13863
13864
13865
13866
13867
13868
13869
13870
13871
13872
13873
13874
13875
13876
13877
13878
13879
13880
13881
13882
13883
13884
13885
13886
13887
13888
13889
13890
13891
13892
13893
13894
13895
13896
13897
13898
13899
13900
13901
13902
13903
13904
13905
13906
13907
13908
13909
13910
13911
13912
13913
13914
13915
1391

In [32]:
pageurl(566)

'https://questions.assemblee-nationale.fr/q16/16-566QE.htm'